# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### Start by importing libraries

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

#!pip install bs4 # Uncomment if bs4 is not installed
from bs4 import BeautifulSoup # import BeautifulSoup

print('Libraries imported.')

### Creates url and saves it as a BeautifulSoup object

In [ ]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(website_url.content,'html.parser')

In [ ]:
#print(soup.prettify()) # Uncomment if you want to see the html data

### Saving raw html data into a panda dataframe

In [ ]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

### Print first 5 entries of data frame and shape

In [ ]:
df.head()

In [ ]:
df.shape

### Creating a new dataframe and reading in csv file for coordinates since geocoder does not work at the moment

In [ ]:
#Creating new df
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

#Importing coordinate df
coordinates = pd.read_csv('Geospatial_Coordinates.csv')

### Iterating through each entry and checking for correct coordinates and adding to new dataframe

In [ ]:
# Iterating through neighborhood df
for i in range(len(df)):

    postalCode = df.iloc[i]['PostalCode']
    borough = df.iloc[i]['Borough'] 
    neighborhood_name = df.iloc[i]['Neighborhood']
    n_lon = None
    n_lat = None

    # Iterating through coordinates df
    for j in range(len(coordinates)):
        if postalCode == coordinates.loc[j][0]:
            n_lat = coordinates.loc[j][1]
            n_lon = coordinates.loc[j][2]

    # Appending data to new df
    neighborhoods = neighborhoods.append({'PostalCode': postalCode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': n_lat,
                                          'Longitude': n_lon}, ignore_index=True)

### New dataframe first 5 entries and shape

In [ ]:
neighborhoods.head()

In [ ]:
neighborhoods.shape